# 라이브러리 불러오기

In [1]:
import os

# numerical calculation & data frames
import numpy as np
import pandas as pd

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so

# statistics
import statsmodels.api as sm

# pandas options
pd.set_option('mode.copy_on_write', True)  # pandas 2.0
pd.options.display.float_format = '{:.2f}'.format  # pd.reset_option('display.float_format')

# NumPy options
np.set_printoptions(precision = 2, suppress=True)  # suppress scientific notation

# For high resolution display
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats("retina")

import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 시각화 함수

In [2]:
def boxplot(df, x, y, color=None, alpha=0.1, marker="<"):
    
    return (
        so.Plot(df, x=x, y=y, color=color)
        .add(so.Dots(alpha=alpha, color=".6"), so.Jitter(), so.Dodge())
        .add(so.Range(), so.Est(errorbar=("pi", 50)), so.Dodge())
        .add(so.Dot(pointsize=8, marker=marker), so.Agg("median"), so.Dodge())
        .scale(color="Dark2")
        .theme({**sns.axes_style("whitegrid")})
    )


def rangeplot(df, x, y, color=None, alpha=0.1, marker="<"):

    return (
        so.Plot(df, x=x, y=y, color=color)
        .add(so.Range(), so.Est(errorbar=("pi", 50)), so.Dodge())
        .add(so.Dot(pointsize=8, marker=marker), so.Agg("median"), so.Dodge())
        .scale(color="Dark2")
        .theme({**sns.axes_style("whitegrid")})
    )

# 랜덤 시드 설정

In [3]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# 데이터 불러오기

In [4]:
data_path = "../data"
train_data = pd.read_csv(os.path.join(data_path, 'train.csv'))
test_data = pd.read_csv(os.path.join(data_path, 'test.csv'))
sample_submission = pd.read_csv(os.path.join(data_path, 'sample_submission.csv'))
school_data = pd.read_csv(os.path.join(data_path, 'schoolinfo.csv'))
park_data = pd.read_csv(os.path.join(data_path, 'parkInfo.csv'))
subway_data = pd.read_csv(os.path.join(data_path, 'subwayInfo.csv'))
interest_data = pd.read_csv(os.path.join(data_path, 'interestRate.csv'))

In [5]:
train_data_copy = train_data.copy()

# Train EDA

- **`index`**: 인덱스 번호
- **`area_m2`**: 면적 (제곱미터)
- **`contract_year_month`**: 계약년월
- **`contract_day`**: 계약일
- **`contract_type`**: 계약 유형(0: 신규, 1:갱신, 2:모름)
- **`floor`**: 층수
- **`built_year`**: 건축 연도
- **`latitude`**: 위도
- **`longitude`**: 경도
- **`age`**: 건물의 나이 (계산된 값)
- **`deposit`**: 전세 실거래가 (타겟 변수)

In [8]:
df_eda = train_data_copy

In [ ]:
print("train, test_data, sample_submission shape : ", train_data.shape, test_data.shape, sample_submission.shape)

In [ ]:
print("train data 변수 요약 정보 확인 ")
display(train_data.describe())

In [ ]:
print("test data 변수 요약 정보 확인 ")
test_data.describe()

In [ ]:
park_data.describe()

## 결측값 확인

In [ ]:
train_data.isna().sum()

## 중복값 확인

중복값을 어떻게 처리할 건지

- 같은 날/ 같은 건물 같은 층수가 같은 가격으로 계약 된 걸 다른 호수가 계약됐을 가능성도 있지 않나?
- 아니면 그냥 중복값은 삭제할까

In [ ]:
train_data[train_data.duplicated()]

In [ ]:
train_data[train_data.drop(columns=['index']).duplicated()]

## 이상치 확인 (boxplot으로 확인)

그래프로 봤을 때는 이상치 없어보임

In [ ]:
# 연속형 변수에 대한 boxplot 그리기
for column in ['area_m2', 'floor', 'age']:
    plt.figure(figsize=(6, 4))
    train_data[[column]].boxplot()
    plt.title(f'Boxplot of {column}')
    plt.show()


# 상관관계

In [ ]:
# 상관계수 계산
corr_matrix = df_eda[['area_m2', 'contract_year_month', 'contract_day', 'contract_type', 'floor',
                      'built_year', 'latitude', 'longitude', 'age', 'deposit']].corr()

# Heatmap 그리기
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap')
plt.show()



## 분포 그래프

In [26]:
def plot_histograms(df):
    continuous_columns = df.select_dtypes(include='number').columns
    
    # Plotting histograms using Seaborn
    for column in continuous_columns:
        plt.figure(figsize=(8, 6))
        sns.histplot(df[column], kde=False, bins=30)
        plt.title(f'Histogram of {column}')
        plt.xlabel(column)
        plt.ylabel('Frequency')
        plt.show()


In [ ]:
df_filtered = df_eda.drop(columns=['index', 'contract_year_month', 'contract_day', 'contract_type'])
plot_histograms(df_filtered)

### 면적

- 면적이 넓을 수록 가격이 높아지는 경향이 있음
- 오래된 집일 수록 면적이 넓은 경향이 있음

In [ ]:
(
    so.Plot(df_eda,x='area_m2',y='deposit')
    .add(so.Dots())
    .add(so.Line(color='orange'),so.PolyFit(5)) # 색상 변경하기
)

In [ ]:
rangeplot(df_eda,x='age_eda',y='area_m2')

### contract_type(0:신규,1:갱신,2:모름)
순서 없음 -> one-hot encoding


- 평균 기준
    - 갱신 > 신규 > 모름 큰 차이는 없음
    - 관련된 정보를 찾아보면 좋을 거 같음
    - 근데 모름 빈도가 많음

In [ ]:
(
    so.Plot(df_eda, x="contract_type")
    .add(so.Bar(), so.Count())  # category type의 변수는 순서가 존재. 
                                # 그렇지 않은 경우 알바벳 순서로. 
).show()

In [ ]:
rangeplot(df_eda,x='contract_type',y='deposit')

### age

- 신식 건물일 수록 비싸지 않을까?
- age가 정확하게 어떤 걸 의미 : 계약일자 빼기 built_year
- age 음수인 경우 : 부호가 잘 못 들어감

In [ ]:
display(train_data[train_data['age'] <= 0]['age'].value_counts())
print("건물 나이가 음수인 경우 :",train_data[train_data['age'] == 0 ]['age'].count())

In [ ]:
train_data[train_data['age']==-3]

In [ ]:
train_data[train_data['age']==-2]

In [ ]:
rangeplot(df_eda,x='age',y='deposit')

- 최신건물이라고 엄청 비싸지는 않음
- 40 - 50 묶어서 저 구간은 확인해보면 좋을 거 같음 -> 건물 크기가 커서 비싸다

### floor

In [ ]:
rangeplot(df_eda,x='floor',y = 'deposit')

In [ ]:
df_eda[df_eda['floor'] <= 0]['floor'].value_counts()

In [ ]:
df_eda[df_eda['floor'] == 0]

### 위도 경도

- 일단 주소로 변환 후 지도 위에 표시하는 방향으로 eda 진행 
    -  지오코딩 너무 오래걸림 일단 지도 위에 표시하는 걸로

- 근처에 학교가 있는지 확인해봄 ,다른 데이터셋과 같이 이용할 수 있음 일단은 학교만 먼저 이용함

### 계약 날짜 - 금리랑 상관 있을 거 같음

In [ ]:
rangeplot(df_eda,x='contract_year_month',y='deposit')

In [ ]:
rangeplot(df_eda,x='contract_day',y='deposit')

# 대회에 필요한 데이터만 활용하기

In [3]:
columns_needed = ['area_m2', 'contract_year_month', 'contract_day', 'contract_type', 'floor', 'latitude', 'longitude', 'deposit']
columns_needed_test = ['area_m2', 'contract_year_month', 'contract_day', 'contract_type', 'floor', 'latitude', 'longitude']
train_data = train_data[columns_needed]
test_data = test_data[columns_needed_test]

# Holdout 데이터셋 설정 (예: 2023년 7월부터 12월까지의 데이터)

In [5]:
holdout_start = 202307
holdout_end = 202312
holdout_data = train_data[(train_data['contract_year_month'] >= holdout_start) & (train_data['contract_year_month'] <= holdout_end)]
train_data = train_data[~((train_data['contract_year_month'] >= holdout_start) & (train_data['contract_year_month'] <= holdout_end))]

# 학습 데이터와 정답 데이터 분리

In [6]:
X_train = train_data.drop(columns=['deposit'])
y_train = train_data['deposit']
X_holdout = holdout_data.drop(columns=['deposit'])
y_holdout = holdout_data['deposit']
X_test = test_data.copy()

# LightGBM 모델 훈련

In [ ]:
lgb_model = lgb.LGBMRegressor(random_state=RANDOM_SEED)
lgb_model.fit(X_train, y_train)

# Holdout 데이터셋에 대한 성능 확인

In [ ]:
lgb_holdout_pred = lgb_model.predict(X_holdout)
lgb_holdout_mae = mean_absolute_error(y_holdout, lgb_holdout_pred)
print("Holdout 데이터셋 성능:")
print(f"LightGBM MAE: {lgb_holdout_mae:.2f}")

------------

# Sample Submission 제출하기

In [27]:
lgb_test_pred = lgb_model.predict(X_test)
sample_submission['deposit'] = lgb_test_pred
sample_submission.to_csv('output.csv', index=False, encoding='utf-8-sig')